# Preprocessing of TAC-based Relation Extraction dataset

This notebook shows how to preprocess data in CONLL format to use it in Knodle framework.

The dataset preproessed here is weakly-supervised dataset built over Knowledge Base Population challenges in the Text Analysis Conference. For development and test purposes the corpus annotated via crowdsourcing and human labeling from KBP is used (Zhang et al. (2017)). The training is done on a weakly-supervised noisy dataset based on TAC KBP corpora (Surdeanu (2013)), also used in Roth (2014). The TAC dataset was annotated with entity pairs extracted from Freebase (Google (2014)) where corresponding relations have been mapped to the 41 TAC relations types as used in the TAC KBP challenges (e.g., per:schools_attended and org:members). The amount of entity pairs per relation was limited to 10.000 and each entity pair is allowed to be mentioned in no more than 500 sentences.

Additionally, if no rule matched a sentence, it was added to the dataset with no_relation label. 

The steps are the following:
- the input data are downloaded from MINIO database: 
    - raw train data in .conll format
    - gold-annotated dev data in .conll format
    - gold-annotated test data in .conll format
    - list of rules with corresponding classes
    - list of classes
- list of rules with corresponding classes is transformed to t matrics
- the non-labelled train data are read from .conll file, annotated with entity pairs. Basing on them, z_train matrix and train data pandas.Dataframe are generated
- the already annotated dev and test data are read from .conll file and stored in pandas.Dataframe format.

## Imports

Firstly, let's make some basic imports

In [44]:
import argparse
import sys
import os
from pathlib import Path
import logging
from typing import Dict
from minio import Minio

import numpy as np
import pandas as pd
import scipy.sparse as sp
from joblib import dump
from tqdm.auto import tqdm

from knodle.trainer.utils import log_section

pd.set_option('display.max_colwidth', -1)
np.set_printoptions(threshold=sys.maxsize)

In [2]:
# define the files names
Z_MATRIX_OUTPUT_TRAIN = "train_rule_matches_z.lib"
Z_MATRIX_OUTPUT_DEV = "dev_rule_matches_z.lib"
Z_MATRIX_OUTPUT_TEST = "test_rule_matches_z.lib"

T_MATRIX_OUTPUT_TRAIN = "mapping_rules_labels.lib"

TRAIN_SAMPLES_OUTPUT = "df_train.lib"
DEV_SAMPLES_OUTPUT = "df_dev.lib"
TEST_SAMPLES_OUTPUT = "df_test.lib"

# define the path to the folder where the data will be stored
data_path = "../../../data_from_minio/TAC"
os.path.join(data_path)

'../../../data_from_minio/TAC'

## Download the dataset

This dataset, as all datasets provided in knodle, could be easily downloaded from Minio database with Minio client. 

In [3]:
def get_conll_config():
    config = {
        "minio_url": "knodle.dm.univie.ac.at",
        "minio_user": "UnM_LN*jSYK74Iz4",
        "minio_pw": "cQOs4|9Dr2_+HuFKneC8@dRgAtrV21i4Dumy",
        "minio_bucket": "knodle",
        "minio_prefix": "datasets/conll",
        "minio_files": [
            "labels.txt",
            "train.conll",
            "dev.conll",
            "test.conll",
            "rules.csv"
        ],
        "data_dir": data_path,
        "num_features": 400,
        "num_classes": 2,
    }
    return config

config = get_conll_config()
client = Minio(config.get("minio_url"), secure=False)

for file in tqdm(config.get("minio_files")):
    client.fget_object(
        bucket_name=config.get("minio_bucket"),
        object_name=os.path.join(config.get("minio_prefix"), file),
        file_path=os.path.join(data_path, file),
    )

In [4]:
# set paths to input data
path_labels = os.path.join(data_path, "labels.txt")
path_rules = os.path.join(data_path, "rules.csv")
path_train_data = os.path.join(data_path, "train.conll")
path_dev_data = os.path.join(data_path, "dev.conll")
path_test_data = os.path.join(data_path, "test.conll")

## Data Preprocessing¶

### Get labels

First, let's read labels from the file with the corresponding label ids.

In [5]:
labels2ids = {}
with open(path_labels, encoding="UTF-8") as file:
    for line in file.readlines():
        relation, relation_enc = line.replace("\n", "").split(",")
        labels2ids[relation] = int(relation_enc)

num_classes = len(labels2ids)

In [6]:
print(labels2ids)

{'per:alternate_names': 0, 'per:date_of_birth': 1, 'per:age': 2, 'per:country_of_birth': 3, 'per:stateorprovince_of_birth': 4, 'per:city_of_birth': 5, 'per:origin': 6, 'per:date_of_death': 7, 'per:country_of_death': 8, 'per:stateorprovince_of_death': 9, 'per:city_of_death': 10, 'per:cause_of_death': 11, 'per:countries_of_residence': 12, 'per:stateorprovinces_of_residence': 13, 'per:cities_of_residence': 14, 'per:schools_attended': 15, 'per:title': 16, 'per:employee_of': 17, 'per:religion': 18, 'per:spouse': 19, 'per:children': 20, 'per:parents': 21, 'per:siblings': 22, 'per:other_family': 23, 'per:charges': 24, 'org:alternate_names': 25, 'org:members': 26, 'org:member_of': 27, 'org:subsidiaries': 28, 'org:political/religious_affiliation': 29, 'org:top_members/employees': 30, 'org:number_of_employees/members': 31, 'org:parents': 32, 'org:founded_by': 33, 'org:founded': 34, 'org:country_of_headquarters': 35, 'org:stateorprovince_of_headquarters': 36, 'org:city_of_headquarters': 37, 'org:

Since we want to preserve samples, where no rule matched, in training set as negative samples, let's heuristically calculate the other class id

In [7]:
other_class_id = max(labels2ids.values()) + 1

### Get rules

Secondly, rules (in our case, entity pairs extracted from Freebase) that are stored in the separate csv file are read and stored.

In [8]:
rules = pd.read_csv(path_rules)
rules.head()

,rule,rule_id,label,label_id
0,Art_Technology_Group ATG,0,org:alternate_names,25
1,Union_Cycliste_Internationale UCI,1,org:alternate_names,25
2,Hanwha 한화,2,org:alternate_names,25
3,Radio_Free_Europe Radio_Liberty,3,org:alternate_names,25
4,Hermès Hermes,4,org:alternate_names,25


In [48]:
df1 = rules[rules.duplicated('rule', keep=False)].groupby('rule')['label'].apply(list).reset_index()
# print(df1[['rule', 'label']])
print(df1)

                                     rule  \
0      23andMe Anne_Wojcicki                
1      23andMe Linda_Avey                   
2      3rd_Baron_Rayleigh Witham            
3      4Kids_Entertainment Alfred_R._Kahn   
4      A.A._Milne Hartfield                 
...                     ...                 
17275  持田_香織 Tokyo_,_Japan                  
17276  柯受良 Shanghai                         
17277  柯受良 Shanghai_,_China                 
17278  柴咲_コウ Tokyo                          
17279  田中_理恵 Sapporo                        

                                                                                                               label  
0      [org:founded_by, org:top_members/employees]                                                                    
1      [org:founded_by, org:top_members/employees]                                                                    
2      [per:city_of_death, per:stateorprovince_of_death]                                              

Let's transform this dataframe into a dictionary with rule to rule ids correspondings. If the same rule corresponds to different relations, it will be stored only once with the one of the relations.

In [9]:
rule2rule_id = dict(zip(rules.rule, rules.rule_id))
num_rules = max(rules.rule_id.values) + 1

### Get rules to classes correspondance matrix

Lastly, we need to know which rule corresponds to which class. This information can be got from t_matrix, which is also stored in rules DataFrame. 

In [10]:
def get_t_matrix(rules: pd.DataFrame, num_classes: int) -> np.ndarray:
    """ Function calculates t matrix (rules x labels) using the known correspondence of relations to decision rules """
    rule_assignments_t = np.empty([rules.rule_id.max() + 1, num_classes])
    for index, row in rules.iterrows():
        rule_assignments_t[row["rule_id"], row["label_id"]] = 1
    return rule_assignments_t

rule_assignments_t = get_t_matrix(rules, num_classes)

## Train data preprocessing

Train data should be annotated with rules we already have. Remember, there are no gold labels (as opposite to dev and test data).

The annotation is done in the following way: 
- the sentences are extracted from conll format
- the tokens labelled as object and subject are checked whether thery are in rules list
- if yes, this sentence is labelled with the corresponding relation
- if not, this sentence is labelled with no_relation label (filter_out_other parameter is set to False)

In [11]:
def count_file_lines(file_name: str) -> int:
    """ Count the number of line in a file """
    with open(file_name) as f:
        return len(f.readlines())

In [12]:
train_data = open(path_train_data)
for i in range(30):
    line = train_data.readline()
    print(line)

#	index	token	subj	subj_type	obj	obj_type	stanford_pos	stanford_ner	stanford_deprel	stanford_head

# id=E0065795:0-pos docid=E0065795:0 reln=org:alternate_names

1	Profile	_	_	_	_	VB	O	advmod	16

2	,	_	_	_	_	,	O	punct	16

3	basic	_	_	_	_	JJ	O	amod	4

4	information	_	_	_	_	NN	O	compound	5

5	ATG	_	_	OBJECT	ORG	NNP	ORG	nsubj	16

6	(	_	_	_	_	-LRB-	O	punct	11

7	Art	SUBJECT	ORGANIZATION	_	_	NNP	ORG	compound	9

8	Technology	SUBJECT	ORGANIZATION	_	_	NNP	ORG	compound	9

9	Group	SUBJECT	ORGANIZATION	_	_	NNP	ORG	nmod	11

10	,	_	_	_	_	,	O	punct	11

11	Inc.	_	_	_	_	NNP	GPE	appos	5

12	,	_	_	_	_	,	O	punct	11

13	NASDAQ	_	_	_	_	NNP	ORG	npadvmod	11

14	:	_	_	_	_	:	O	punct	13

15	)	_	_	_	_	-RRB-	O	punct	11

16	makes	_	_	_	_	VBZ	O	ROOT	16

17	software	_	_	_	_	NN	O	dobj	16

18	and	_	_	_	_	CC	O	cc	16

19	delivers	_	_	_	_	VBZ	O	conj	16

20	e	_	_	_	_	NN	O	nmod	22

21	-	_	_	_	_	HYPH	O	punct	22

22	commerce	_	_	_	_	NN	O	nmod	26

23	and	_	_	_	_	CC	O	cc	22

24	Web	_	_	_	_	NN	O	compound	25

25	marketing	_	_	_	

In [13]:
def annotate_conll_data_with_lfs(conll_data: str, rule2rule_id: Dict, filter_out_other: bool = True) -> pd.DataFrame:
    num_lines = count_file_lines(conll_data)
    processed_lines = 0
    samples, rules, enc_rules = [], [], []
    with open(conll_data, encoding='utf-8') as f:
        for line in f:
            processed_lines += 1
            line = line.strip()
            if line.startswith("# id="):  # Instance starts
                sample = ""
                subj, obj = {}, {}
            elif line == "":  # Instance ends
                if len(list(subj.keys())) == 0 or len(list(obj.keys())) == 0:
                    continue
                if min(list(subj.keys())) < min(list(obj.keys())):
                    rule = "_".join(list(subj.values())) + " " + "_".join(list(obj.values()))
                else:
                    rule = "_".join(list(subj.values())) + " " + "_".join(list(obj.values()))
                if rule in rule2rule_id.keys():
                    samples.append(sample)
                    rules.append(rule)
                    rule_id = rule2rule_id[rule]
                    enc_rules.append(rule_id)
                elif not filter_out_other:
                    samples.append(sample)
                    rules.append(None)
                    enc_rules.append(None)
                else:
                    continue
            elif line.startswith("#"):  # comment
                continue
            else:
                splitted_line = line.split("\t")
                token = splitted_line[1]
                if splitted_line[2] == "SUBJECT":
                    subj[splitted_line[0]] = token
                    sample += " " + token
                elif splitted_line[4] == "OBJECT":
                    obj[splitted_line[0]] = token
                    sample += " " + token
                else:
                    sample += " " + token
            if processed_lines % (int(round(num_lines / 10))) == 0:
                print(f"Processed {processed_lines / num_lines * 100 :0.0f}% of {conll_data.split('/')[-1]} file")
                
    print(f"Preprocessing of {conll_data.split('/')[-1]} file is finished")
    return pd.DataFrame.from_dict({"samples": samples, "rules": rules, "enc_rules": enc_rules})

In [14]:
train_data = annotate_conll_data_with_lfs(path_train_data, rule2rule_id, False)

Processed 10% of train.conll file
Processed 20% of train.conll file
Processed 30% of train.conll file
Processed 40% of train.conll file
Processed 50% of train.conll file
Processed 60% of train.conll file
Processed 70% of train.conll file
Processed 80% of train.conll file
Processed 90% of train.conll file
Preprocessing of train.conll file is finished


After that we could build a z_matrix for train data and save it as a sparse matrix .

In [15]:
def get_z_matrix(data: pd.DataFrame, num_rules: int) -> np.ndarray:
    """
    Function calculates the z matrix (samples x rules)
    data: pd.DataFrame (samples, matched rules, matched rules id )
    output: sparse z matrix
    """
    data_without_nan = data.reset_index().dropna()
    z_matrix_sparse = sp.csr_matrix(
        (
            np.ones(len(data_without_nan['index'].values)),
            (data_without_nan['index'].values, data_without_nan['enc_rules'].values)
        ),
        shape=(len(data.index), num_rules)
    )
    return z_matrix_sparse

In [16]:
train_rule_matches_z = get_z_matrix(train_data, num_rules)

## Dev & Test data preprocessing¶

The dev and test data are to be simply read from the data without any additional annotation since the gold label are known for them.  

In [17]:
def get_conll_data_with_labels(
        conll_data: str, rule2rule_id: Dict, labels2ids: dict, other_class_id: int = None
) -> pd.DataFrame:
    """
    Processing of TACRED dataset. The function reads the .conll input file, extract the samples and the labels as well
    as argument pairs, which are saved as decision rules.
    :param conll_data: input data in .conll format
    :param rule2rule_id: corresponding of rules to rules ids
    :param labels2ids: dictionary of label - id corresponding
    :param other_class_id: id of other_class_label
    :return: DataFrame with columns "samples" (extracted sentences), "rules" (entity pairs), "enc_rules" (entity pairs
            ids), "labels" (original labels)
    """

    num_lines = count_file_lines(conll_data)
    processed_lines = 0

    samples, labels, rules, enc_rules = [], [], [], []
    with open(conll_data, encoding='utf-8') as f:
        for line in f:
            processed_lines += 1
            line = line.strip()
            if line.startswith("# id="):  # Instance starts
                sample = ""
                subj, obj = {}, {}
                label = labels2ids.get(line.split(" ")[3][5:], other_class_id)
            elif line == "":  # Instance ends
                if min(list(subj.keys())) < min(list(obj.keys())):
                    rule = "_".join(list(subj.values())) + " " + "_".join(list(obj.values()))
                else:
                    rule = "_".join(list(subj.values())) + " " + "_".join(list(obj.values()))

                if rule in rule2rule_id.keys():
                    samples.append(sample)
                    labels.append(label)
                    rules.append(rule)
                    rule_id = rule2rule_id[rule]
                    enc_rules.append(rule_id)

                else:
                    samples.append(sample)
                    labels.append(label)
                    rules.append(None)
                    enc_rules.append(None)

            elif line.startswith("#"):  # comment
                continue
            else:
                splitted_line = line.split("\t")
                token = splitted_line[1]
                if splitted_line[2] == "SUBJECT":
                    subj[splitted_line[0]] = token
                    sample += " " + token
                elif splitted_line[4] == "OBJECT":
                    obj[splitted_line[0]] = token
                    sample += " " + token
                else:
                    sample += " " + token
                    
            if processed_lines % (int(round(num_lines / 10))) == 0:
                    print(f"Processed {processed_lines / num_lines * 100 :0.0f}% of {conll_data.split('/')[-1]} file")
    print(f"Preprocessing of {conll_data.split('/')[-1]} file is finished")
    return pd.DataFrame.from_dict({"samples": samples, "rules": rules, "enc_rules": enc_rules, "labels": labels})

In [18]:
dev_data = get_conll_data_with_labels(path_dev_data, rule2rule_id, labels2ids, other_class_id)
test_data = get_conll_data_with_labels(path_test_data, rule2rule_id, labels2ids, other_class_id)

Processed 10% of dev.conll file
Processed 20% of dev.conll file
Processed 30% of dev.conll file
Processed 40% of dev.conll file
Processed 50% of dev.conll file
Processed 60% of dev.conll file
Processed 70% of dev.conll file
Processed 80% of dev.conll file
Processed 90% of dev.conll file
Preprocessing of dev.conll file is finished
Processed 10% of test.conll file
Processed 20% of test.conll file
Processed 30% of test.conll file
Processed 40% of test.conll file
Processed 50% of test.conll file
Processed 60% of test.conll file
Processed 70% of test.conll file
Processed 80% of test.conll file
Processed 90% of test.conll file
Processed 100% of test.conll file
Preprocessing of test.conll file is finished


z_matrix could be build with the same function as we used for building the z_matrix for train data.

In [19]:
dev_rule_matches_z = get_z_matrix(dev_data, num_rules)
test_rule_matches_z = get_z_matrix(test_data, num_rules)

## Statistics

Let's analyse the data we collected.

In [27]:
print(f"Number of samples in train set: {len(train_data)}")
print(f"Number of samples in dev set: {len(dev_data)}")
print(f"Number of samples in test set: {len(test_data)}")
print(f"WTF clarify: {len(rules)}")
print(f"Number of rules: {num_rules}")
print(f"Dimension of t matrix: {rule_assignments_t.shape}")

Number of samples in train set: 1937211
Number of samples in dev set: 5368
Number of samples in test set: 18660
WTF clarify: 206073
Number of rules: 182292
Dimension of t matrix: (182292, 41)


## Save files

... and we save all the data we got. 

In [20]:
Path(os.path.join(data_path, "preprocessed")).mkdir(parents=True, exist_ok=True)

dump(sp.csr_matrix(rule_assignments_t), os.path.join(data_path, "preprocessed", T_MATRIX_OUTPUT_TRAIN))

dump(train_data, os.path.join(data_path, "preprocessed", TRAIN_SAMPLES_OUTPUT))
dump(train_rule_matches_z, os.path.join(data_path, "preprocessed", Z_MATRIX_OUTPUT_TRAIN))

dump(dev_data, os.path.join(data_path, "preprocessed", DEV_SAMPLES_OUTPUT))
dump(dev_rule_matches_z, os.path.join(data_path, "preprocessed", Z_MATRIX_OUTPUT_DEV))

dump(test_data, os.path.join(data_path, "preprocessed", TEST_SAMPLES_OUTPUT))
dump(test_rule_matches_z, os.path.join(data_path, "preprocessed", Z_MATRIX_OUTPUT_TEST))

['../../../data_from_minio/TAC/preprocessed/test_rule_matches_z.lib']

## Finish